# Assignment 1

In [63]:
!pip install wordcloud

In [64]:
import re
import pandas as pd
import nltk

In [65]:
nltk.download("stopwords")
nltk.download("words")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gener\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gener\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gener\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gener\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [66]:
df = pd.read_csv("A1_dataset.csv")

In [67]:
df.head()

,LABEL,DATE_TIME,TEXT
0,0,Fri Jun 05 14:26:50 2009,About to get threaded and scared
1,1,Thu May 14 10:13:55 2009,@awaisnaseer I like Shezan Mangooo too!!! I ha...
2,1,Fri Jun 05 21:02:20 2009,worked on my car after work. showering then go...
3,1,Sun Jun 14 22:25:52 2009,@Marama Actually we start this afternoon! I w...
4,1,Sun May 31 00:42:12 2009,@gfalcone601 Aww Gi.don't worry.we'll vote for...


### II) Text Preprocessing

In [68]:
from nltk.tokenize import TweetTokenizer

def tokenization(text):
    twt_tk = TweetTokenizer()
    return ' '.join(twt_tk.tokenize(text))

In [112]:
# Can use Jaccard distance or edit distance to do this
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.corpus import words

wrd = words.words()

# Spelling correction using jaccard distance
def spelling_correction_jaccard(text):
    res = []
    for curr in text:
        if curr not in wrd:
            crrct_words = []
            for w in wrd:
                crrct_words.append([jaccard_distance(set(w), set(curr)), w])
            crrct_words = sorted(crrct_words, key = lambda x: x[0])
            res.append(crrct_words[0][1][0])
        else:
            res.append(curr)
    return ' '.join(res)

# Spelling correction using edit distance
def spelling_correction_edit(text):
    res = []
    for curr in text:
        if curr not in wrd:
            crrct_words = []
            for w in wrd:
                crrct_words.append([edit_distance(w, curr), w])
            crrct_words = sorted(crrct_words, key = lambda x: x[0])
            res.append(crrct_words[0][1])
        else:
            res.append(curr)
    return ' '.join(res)

In [113]:
spelling_correction_jaccard(["helllo", "good"]), spelling_correction_edit(["helllo", "good"])

#As we can see from the results below that the corrected words we get using edit distance are
#far better than the ones we get from jaccard distance

('h good', 'hello good')

In [71]:
#Here we can either do lemmatization or stemming, doing both at the same time will not be useful

from nltk.stem import WordNetLemmatizer, PorterStemmer

def lemmatization_text(text):
    wn = WordNetLemmatizer()
    res = []
    for each in text:
        lemmatized_text = wn.lemmatize(each)
        res.append(lemmatized_text)
    return res

def stemming_text(text):
    res = []
    ps = PorterStemmer()
    for each in text:
        stemmed_text = ps.stem(each)
        res.append(stemmed_text)
    return res

In [72]:
lemmatization_text(["dogs", "becomes"]), stemming_text(["dogs", "becomes"])

(['dog', 'becomes'], ['dog', 'becom'])

In [73]:
import string

def remove_punctuations(text):
    puncts = string.punctuation
    return ''.join(re.split("[" + puncts + "]", text))

In [74]:
print("Punctuations: ", string.punctuation)
remove_punctuations("lesgoooo!!!!!?!??!! whooohooo!")

Punctuations:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


'lesgoooo whooohooo'

In [75]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    pattern = re.compile(r'\b(' + (r'|'.join(stopwords.words("english"))) + r')\b\s*')
    return pattern.sub('', text)

In [76]:
remove_stopwords("the is when am i the when the when name is")

'name '

In [77]:
def remove_whitespaces(text):
    return re.sub(r'\s*\s', ' ', text).split()

In [78]:
remove_whitespaces("whoa a    s f")

['whoa', 'a', 's', 'f']

In [79]:
def remove_url_html(text):
    text = text.lower()
    text = re.sub('(?:http://|https://)?[A-Za-z0-9_]+\.[A-Za-z0-9_]{2,}[\.A-Za-z0-9_]*[/?[A-Za-z0-9_]*]*\.?[A-Za-z0-9_]*\\b', '', text)
    return text

In [80]:
remove_url_html('Something so beautiful I want to share with my tweeter friends  http://www.spiritisup.com/ahuginyourinboxgmb.html')

'something so beautiful i want to share with my tweeter friends  '

### III) Visualization

In [115]:
# Here we prefer to perform lemmatization over stemming so that the words don't lose their inherent meaning
# and we perform spelling correction using the edit_distance metric in nltk, this is mainly because of the results shown above
# comparing the jaccard distance and the edit distance method.

def preprocess_text(text):
    text = tokenization(text)
    text = remove_url_html(text)
    text = remove_punctuations(text)
    text = remove_whitespaces(text)
    text = spelling_correction_edit(text)
    text = remove_stopwords(text)
    text = ''.join(lemmatization_text(text))
    return text

In [ ]:
df["TEXT"] = df["TEXT"].apply(lambda row: preprocess_text(row))

In [ ]:
preprocessed_text = ' '.join(df["TEXT"])

In [103]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
wordcloud = WordCloud(width = 1500, height = 720,
                background_color ='black',
                min_font_size = 10).generate(preprocessed_text)

In [ ]:
plt.axis("off")
plt.imshow(wordcloud, interpolation="bilinear")
plt.show()